In [1]:
import weatherAPI
import pandas as pd
import numpy as np
import datetime

In [2]:
weatherAPI.load2('20200801','20200801','90')

,location,date,avg_temperature(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,속초,20200801,26.3,4.1,0.7,79.1


In [3]:
AI_occur=pd.read_excel("./aiai/data/HPAIOccurlist_v2.xlsx")

In [4]:
AI_occur=AI_occur.rename(columns={'사육_x000D_\n두수':'사육수'})

In [5]:
AI_occur.set_index('번호', drop=True, inplace=True)
AI_occur.reset_index(drop=True, inplace=True)

In [6]:
AI_occur.head()

,주소,축종,사육수,진단일,발생일,지역코드
0,전라남도 장흥군 장평면 양촌리,오리-육용오리,9500,20210407,20210406,260
1,전라남도 장흥군 장평면 양촌리,오리-육용오리,15500,20210407,20210406,260
2,전라남도 나주시 산포면 등수리,오리-육용오리,19000,20210324,20210323,156
3,전라남도 장흥군 장평면 기동리,오리-육용오리,11800,20210323,20210322,260
4,전라남도 장흥군 장평면 기동리,오리-육용오리,20000,20210322,20210321,260


In [7]:
AI_occur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   주소      1122 non-null   object
 1   축종      1122 non-null   object
 2   사육수     1122 non-null   int64 
 3   진단일     1122 non-null   int64 
 4   발생일     1122 non-null   int64 
 5   지역코드    1122 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 52.7+ KB


In [8]:
AI_occur['진단일']=AI_occur['진단일'].astype('str')
AI_occur['발생일']=AI_occur['발생일'].astype('str')
AI_occur['지역코드']=AI_occur['지역코드'].astype('str')

In [9]:
AI_occur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   주소      1122 non-null   object
 1   축종      1122 non-null   object
 2   사육수     1122 non-null   int64 
 3   진단일     1122 non-null   object
 4   발생일     1122 non-null   object
 5   지역코드    1122 non-null   object
dtypes: int64(1), object(5)
memory usage: 52.7+ KB


### 관계성 분석1
 1. 축종별 사육수에 따른 시계열적 분석
     <br>1.1 사육수를 제외하고 모든 열이 다 같을 경우 사육수만 sum한 후 groupby
     <br>1.2 축종별마다 해당하는 날씨 데이터 결합
     <br>1.3 축종별 분류하기

In [10]:
dup_AI=AI_occur.groupby(['주소','축종','진단일','발생일','지역코드']).sum()

In [11]:
dup_AI.reset_index(inplace=True)

In [12]:
dup_AI.head()  # 사육수 sum()

,주소,축종,진단일,발생일,지역코드,사육수
0,강원도 원주시 귀래면 주포리,닭-산란계,20210224,20210223,114,170000
1,강원도 인제군 기린면 북리,닭-산란계,20170107,20170103,211,28500
2,강원도 철원군 갈말읍 강포리,닭-산란계,20161203,20161130,95,30000
3,강원도 철원군 갈말읍 강포리,닭-산란계,20161212,20161209,95,16000
4,강원도 춘천시 사북면 지촌리,닭-산란계,20080507,20080507,101,96


In [15]:
# 축종별 날씨data와 결합
weather_list=['location','date','avg_temperature(C)','daily_rain(mm)','avg_wind(m/s)','avg_r_humidity(%)']
for name in weather_list:
     dup_AI[name]=np.nan
for i in range(len(dup_AI)):
    start=dup_AI.loc[i,'발생일']
    w_data=weatherAPI.load2(start, start, dup_AI.loc[i,'지역코드'])
    try:
        for col in weather_list:
            dup_AI.loc[i,col]=w_data.loc[0,col]
    except:
        print('error index:', i)

API error
error index: 9
API error
error index: 23
API error
error index: 24
API error
error index: 25
API error
error index: 235
API error
error index: 236
API error
error index: 237
API error
error index: 238
API error
error index: 239
API error
error index: 240
API error
error index: 241
API error
error index: 242
API error
error index: 243
API error
error index: 244
API error
error index: 245
API error
error index: 246
API error
error index: 247
API error
error index: 249
API error
error index: 250
API error
error index: 251
API error
error index: 252
API error
error index: 253
API error
error index: 254
API error
error index: 255
API error
error index: 268
API error
error index: 269
API error
error index: 272
API error
error index: 279
API error
error index: 280
API error
error index: 283
API error
error index: 284
API error
error index: 433
API error
error index: 434
API error
error index: 435
API error
error index: 436
API error
error index: 437
API error
error index: 438
API er

In [16]:
dup_AI

,주소,축종,진단일,발생일,지역코드,사육수,location,date,avg_temperature(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,강원도 원주시 귀래면 주포리,닭-산란계,20210224,20210223,114,170000,원주,20210223,1.9,0.0,1.4,28.0
1,강원도 인제군 기린면 북리,닭-산란계,20170107,20170103,211,28500,인제,20170103,1.2,0.0,2.2,60.1
2,강원도 철원군 갈말읍 강포리,닭-산란계,20161203,20161130,95,30000,철원,20161130,3.2,1.2,0.9,72.1
3,강원도 철원군 갈말읍 강포리,닭-산란계,20161212,20161209,95,16000,철원,20161209,-0.1,1.7,1.6,68.9
4,강원도 춘천시 사북면 지촌리,닭-산란계,20080507,20080507,101,96,춘천,20080507,13.2,17.0,0.6,66.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1029,충청북도 청주시 흥덕구 오송읍 오송리,닭-육계,20161206,20161204,131,1500,청주,20161204,4.3,0.0,0.6,63.3
1030,충청북도 충주시 대소원면 만정리,닭-토종닭,20161208,20161206,127,100,충주,20161206,-1.1,0.0,2.1,49.1
1031,충청북도 충주시 동량면 대전리,오리-종오리,20210203,20210202,133,7600,대전,20210202,-2.4,0.0,2.5,46.9
1032,충청북도 충주시 소태면 주치리,닭-산란계,20210313,20210311,127,550000,충주,20210311,8.7,0.0,0.8,51.6


In [17]:
dup_AI.to_csv("HAPI_Weather_Occur.csv", encoding='utf-8-sig', index=False)

In [18]:
species_list={}
for n1, group in dup_AI.groupby('축종'):
    group=group.sort_values('발생일') # 발생일순 정렬
    group.reset_index(drop=True, inplace=True)
    species_list[n1]=group

In [19]:
species_list.keys()

dict_keys(['개-기타', '거위-비분류', '고양이-코리아숏헤어', '관상조류-기러기', '관상조류-비분류', '꿩', '닭', '닭-기타', '닭-백세미', '닭-비분류', '닭-산란계', '닭-삼계', '닭-원종계', '닭-육계', '닭-일괄', '닭-종계', '닭-토종닭', '메추리', '메추리-비분류', '야생오리-가창오리', '야생오리-청둥오리', '야생조류-고니', '야생조류-기러기', '야생조류-기타', '야생조류-논병아리', '야생조류-물닭', '야생조류-백로', '야생조류-왜가리', '야생조류-큰기러기', '오리', '오리-기타', '오리-비분류', '오리-산란오리', '오리-육용오리', '오리-종오리', '칠면조', '칠면조-비분류', '타조-비분류'])

In [21]:
species_list['고양이-코리아숏헤어']

,주소,축종,진단일,발생일,지역코드,사육수,location,date,avg_temperature(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,경기도 포천시 영북면 자일리,고양이-코리아숏헤어,20161231,20161226,98,14,동두천,20161226,1.5,0.0,0.9,74.6
1,경기도 포천시 영북면 자일리,고양이-코리아숏헤어,20170102,20161230,98,7,동두천,20161230,-4.3,0.0,0.9,54.4


In [4]:
data = pd.read_csv('aiai/data_csv/HAPI_Weather_Occur.csv')

In [5]:
data

,주소,축종,진단일,발생일,지역코드,사육수,location,date,avg_temperature(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,강원도 원주시 귀래면 주포리,닭-산란계,20210224,20210223,114,170000,원주,20210223.0,1.9,0.0,1.4,28.0
1,강원도 인제군 기린면 북리,닭-산란계,20170107,20170103,211,28500,인제,20170103.0,1.2,0.0,2.2,60.1
2,강원도 철원군 갈말읍 강포리,닭-산란계,20161203,20161130,95,30000,철원,20161130.0,3.2,1.2,0.9,72.1
3,강원도 철원군 갈말읍 강포리,닭-산란계,20161212,20161209,95,16000,철원,20161209.0,-0.1,1.7,1.6,68.9
4,강원도 춘천시 사북면 지촌리,닭-산란계,20080507,20080507,101,96,춘천,20080507.0,13.2,17.0,0.6,66.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1029,충청북도 청주시 흥덕구 오송읍 오송리,닭-육계,20161206,20161204,131,1500,청주,20161204.0,4.3,0.0,0.6,63.3
1030,충청북도 충주시 대소원면 만정리,닭-토종닭,20161208,20161206,127,100,충주,20161206.0,-1.1,0.0,2.1,49.1
1031,충청북도 충주시 동량면 대전리,오리-종오리,20210203,20210202,133,7600,대전,20210202.0,-2.4,0.0,2.5,46.9
1032,충청북도 충주시 소태면 주치리,닭-산란계,20210313,20210311,127,550000,충주,20210311.0,8.7,0.0,0.8,51.6
